## 주제 
- 위성 소형화, 저궤도 위성, 광학위성

In [27]:
"""
담당자별 에이전트:
- 현준: startup_search_agent (스타트업 탐색)
- 승연: tech_summary_agent (기술 요약)
- 민주: market_eval_agent (시장성 평가)
- 경남: investor_insight_agent (투자자 인사이트)

+ investment_decision_agent (투자 판단)
"""

'\n담당자별 에이전트:\n- 현준: startup_search_agent (스타트업 탐색)\n- 승연: tech_summary_agent (기술 요약)\n- 민주: market_eval_agent (시장성 평가)\n- 경남: investor_insight_agent (투자자 인사이트)\n\n+ investment_decision_agent (투자 판단)\n'

In [28]:
# API KEY Loading
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
from typing import Annotated, TypedDict, Sequence, Literal
from pydantic import BaseModel

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import create_react_agent

from langchain.tools import tool
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA

from decision_module import investment_decision_agent

In [ ]:
# ====== AgentState (업데이트된 스키마) ======
class AgentState(TypedDict):
    startup_name: str
    startup_info: dict
    tech_summary: str
    market_analysis: str
    investment_decision: int     # 투자=1, 비투자=0
    decision_reason: str         # 판단 근거 요약 문자열
    report: str                  # 상세 리포트
    iteration_count: int


## 스타트업 탐색 에이전트 (담당자: 현준)
1. 웹서치로 스타트업 기본 정보 수집


In [31]:
def startup_search_agent(state: AgentState) -> AgentState:
    # TODO: web_search 또는 web_fetch로 스타트업 정보 수집
    # 예시: results = web_search("small satellite startup 2024")

    # 수집한 정보를 state에 저장
    state["startup_name"] = "Example Satellite Startup"
    state["startup_info"] = {
        "description": "소형 위성 기반 지구 관측 서비스",
        "founding_year": 2020,
        "technology": "CubeSat, 광학위성",
        "website": "https://example.com",
    }

    return state

## 기술 요약 에이전트 (담당자: 승연)

In [ ]:
def tech_summary_agent(state: AgentState) -> AgentState:
    """승연 담당: 기술력 핵심 요약"""

    state["tech_summary"] = f"{company_name}의 기술 분석 결과 (승연 담당)"

    return state

## 시장성 평가 에이전트 (민주)

In [ ]:
def market_eval_agent(state: AgentState) -> AgentState:
    """민주 담당: 시장 성장성, 수요 분석"""

    company_name = state["startup_name"]
    state["market_analysis"] = f"{company_name}의 시장 분석 결과 (민주 담당)"

    return state

## 투자 판단 에이전트


In [ ]:
# def investment_decision_agent(state: AgentState) -> AgentState:
#     """경남 담당: 투자자 인사이트 분석 + Scorecard Method 기반 투자 판단"""

#     company_name = state["startup_name"]
#     tech_summary = state["tech_summary"]
#     market_analysis = state["market_analysis"]

#     llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#     combined_prompt = f"""
#     당신은 세계적인 벤처캐피탈리스트입니다.
    
#     회사: {company_name}
#     기술 분석: {tech_summary}
#     시장 분석: {market_analysis}
    
#     ## Part 1: 투자자 인사이트
#     다음 관점에서 분석하세요:
#     1. 비전과 시장 타이밍
#     2. 팀의 실행력 가능성
#     3. 스케일업 잠재력
#     4. 경쟁 우위 지속가능성
    
#     ## Part 2: Scorecard Method 평가
#     각 항목을 50-150% 척도로 평가:
#     - 제품/기술: 15-20%
#     - 시장 기회: 20-25%
#     - 팀: 25-30%
#     - 경쟁환경: 10-15%
    
#     형식:
#     [투자자 인사이트]
#     - 비전과 타이밍: ...
#     - 실행력: ...
#     - 스케일업: ...
#     - 경쟁우위: ...
    
#     [Scorecard 평가]
#     기술: X%
#     시장: Y%
#     팀: Z%
#     경쟁: W%
    
#     [최종 결론]
#     invest 또는 hold
#     """

#     result = llm.invoke(combined_prompt)
#     evaluation_text = result.content

#     if "invest" in evaluation_text.lower() and "hold" not in evaluation_text.lower():
#         state["investment_decision"] = "invest"
#     else:
#         state["investment_decision"] = "hold"

#     state["iteration_count"] = state.get("iteration_count", 0) + 1

#     return state


# # %%
# def should_continue(state: AgentState) -> str:
#     """조건 분기"""
#     if state["investment_decision"] == "invest":
#         return "report_writer"
#     elif state["iteration_count"] >= 3:
#         return "report_writer"
#     return "startup_search"

## 보고서 생성 에이전트

In [ ]:
def report_writer_agent(state: AgentState) -> AgentState:
    """
    결과 요약 보고서 생성
    """
    print("=== 보고서 생성 에이전트 실행 ===")

    company_name = state["startup_name"]
    company_info = state["startup_info"]
    tech_summary = state["tech_summary"]
    market_analysis = state["market_analysis"]
    investment_decision = state["investment_decision"]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    report_prompt = f"""
    당신은 전문 벤처 투자 분석가입니다.
    아래 분석 결과를 바탕으로 다음 형식에 맞춰 투자 평가 보고서를 작성하세요.
    
    === 분석 정보 ===
    
    회사명: {company_name}
    회사 정보: {company_info}
    
    기술력 분석:
    {tech_summary}
    
    시장성 분석:
    {market_analysis}
    
    
    투자 판단: {investment_decision}
    
    === 보고서 형식 ===
    
    # 위성 스타트업 투자 평가 보고서
    
    ## Executive Summary
    
    ## 1. 회사 개요
    
    ## 2. 기술력 평가
    ### 2.1 핵심 기술
    ### 2.2 기술적 강점
    ### 2.3 기술 리스크
    ### 2.4 Scorecard 평가
    
    ## 3. 시장성 분석
    ### 3.1 시장 규모 및 성장률
    ### 3.2 시장 기회
    ### 3.3 Scorecard 평가
    
    ## 4. 경쟁 환경
    
    ## 5. 투자 의견
    ### 5.1 최종 판단
    ### 5.2 투자 근거
    ### 5.3 주요 리스크
    
    ## 6. 결론
    """

    result = llm.invoke(report_prompt)
    state["report"] = result.content

    print("보고서 생성 완료")
    return state

## LangGraph 구성

In [ ]:
def create_workflow():
    graph = StateGraph(AgentState)

    graph.add_node("startup_search", startup_search_agent)
    graph.add_node("tech_summary", tech_summary_agent)
    graph.add_node("market_eval", market_eval_agent)
    graph.add_node("investment_decision", investment_decision_agent)
    graph.add_node("report_writer", report_writer_agent)

    graph.set_entry_point("startup_search")

    graph.add_edge("startup_search", "tech_summary")
    graph.add_edge("tech_summary", "market_eval")
    graph.add_edge("market_eval", "investment_decision")

    graph.add_conditional_edges(
        "investment_decision",
        should_continue,
        {"report_writer": "report_writer", "startup_search": "startup_search"},
    )

    graph.set_finish_point("report_writer")

    return graph.compile()